### 1. elasticsearch clinent 연결

In [1]:
from elasticsearch import Elasticsearch
# from datasets import load_from_disk

es = Elasticsearch('http://localhost:9200')
es.info()

/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/1599703845.py:5: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.info()


ObjectApiResponse({'name': 'single-node', 'cluster_name': 'fastcampus', 'cluster_uuid': 'oliiLtTLTAyiKLsERJNtFQ', 'version': {'number': '7.17.4', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '79878662c54c886ae89206c685d9f1051a9d6411', 'build_date': '2022-05-18T18:04:20.964345128Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [2]:
# 실행된 node 정보
!curl -XGET localhost:9200

{
  "name" : "single-node",
  "cluster_name" : "fastcampus",
  "cluster_uuid" : "oliiLtTLTAyiKLsERJNtFQ",
  "version" : {
    "number" : "7.17.4",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "79878662c54c886ae89206c685d9f1051a9d6411",
    "build_date" : "2022-05-18T18:04:20.964345128Z",
    "build_snapshot" : false,
    "lucene_version" : "8.11.1",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


### 2. index & mapping 생성

In [46]:
# index명
INDEX = 'search_index' 
# mapping 정보 ->  대부분 json으로 저장해서 불러와서 쓴다. 
index_settings = {
    "settings" : {
        "number_of_shards" : 2,
        "number_of_replicas" : 1 
    },
    "mappings" : {
        "properties" : {
            "id" : {"type":"text"},
            "link" : {"type":"text"},
            "title" : {"type":"text"},
            "latex" : {"type":"text"},
            "embedding" : {"type":"dense_vector", 
                           "dims": 512}, # 벡터를 넣기 위해서는 차원을 지정해야 한다. 
            "@timestamp" : {"type":"date"}
        }
    }
}
# mapping 정보를 지정하지 않는 경우 mapping이 자동으로 생성되지만 데이터 타입을 지정하지 못한다. 

# 생성된 인덱스는 다시 create하지 않게 
if es.indices.exists(index=INDEX):
	pass
else:
	es.indices.create(index=INDEX, body = index_settings)

/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/3033068698.py:24: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if es.indices.exists(index=INDEX):
/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/3033068698.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=INDEX, body = index_settings)
/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/3033068698.py:27: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.

In [47]:
# es.indices.delete(index='search_index') # 잘못 생성된 index 없애기

In [48]:
es.indices.get_settings() # 설정 확인

/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/979579740.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.get_settings() # 설정 확인


ObjectApiResponse({'search_index': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '2', 'provided_name': 'search_index', 'creation_date': '1662365207963', 'number_of_replicas': '1', 'uuid': 'MJQKsuuvSQC4J-F-9txm7A', 'version': {'created': '7170499'}}}}})

### 3. latex 임베딩 생성

In [43]:
# 수집된 데이터 불러오기 -> 데이터 크롤링 파트와 연동 (바로 클라우드에 연동될 수 있게 변경)
import json 
with open("anno.json", "r", encoding="utf-8") as json_file:
    latex_data = json.load(json_file)
print(latex_data['latex_anno'][0]['id'])
print(latex_data['latex_anno'][0]['latex'][0])
print(latex_data['latex_anno'][0]['title'])

contexts = list()
for v in latex_data['latex_anno']:
    id, link, title = v['id'], v['link'], v['title']
    for latex in v['latex']:
        contexts.append((latex, id, link, title))

print(f"Lengths of unique contexts : {len(contexts)}")

dc9fd2c45824afc
u,v\in \Gamma (\mathrm {T} M)
리만 곡률 텐서
Lengths of unique contexts : 289


(추가 사항) latex의 임베딩을 하나의 property로 추가

### 3. 데이터 삽입

In [36]:
for idx, data in enumerate(contexts):
    latex, id, link, title = data
    body = {'latex': latex, 'id': id, 'link': link, 'title': title}
    # print(body)
    es.index(index=INDEX, id=id, body=body)

/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/4251627907.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX, id=id, body=body)
/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/4251627907.py:5: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index=INDEX, id=id, body=body)


In [37]:
es.get(index=INDEX, id='dc9fd2c45824afc')

/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/1279614632.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.get(index=INDEX, id='dc9fd2c45824afc')


ObjectApiResponse({'_index': 'search_index', '_type': '_doc', '_id': 'dc9fd2c45824afc', '_version': 50, '_seq_no': 124, '_primary_term': 1, 'found': True, '_source': {'latex': "(\\iota \\iota ',\\iota ''\\iota ''')", 'id': 'dc9fd2c45824afc', 'link': 'https://ko.wikipedia.org/wiki/%EB%A6%AC%EB%A7%8C_%EA%B3%A1%EB%A5%A0', 'title': '리만 곡률 텐서'}})

In [41]:
body = {
		'query': {
				'match': {
						'latex': 'u,v\in \Gamma (\mathrm {T} M)'
				}
		}
}
res = es.search(index=INDEX, body=body)
print(res)

/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/639292111.py:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index=INDEX, body=body)


{'took': 519, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 9.281366, 'hits': [{'_index': 'search_index', '_type': '_doc', '_id': 'f49097b47f255c6', '_score': 9.281366, '_source': {'latex': '\\scriptstyle \\mathrm {distance\\ in\\ parsecs} ={\\frac {1000}{\\mathrm {parallax\\ in\\ milliarcseconds} }}', 'id': 'f49097b47f255c6', 'link': 'https://ko.wikipedia.org/wiki/HR_8799_b', 'title': 'HR 8799 b'}}, {'_index': 'search_index', '_type': '_doc', '_id': '77e840f5bef1b29', '_score': 6.7158313, '_source': {'latex': '{\\mathcal {L}}={{\\psi }}(i\\gamma ^{\\mu }\\partial _{\\mu }-m)\\psi -{\\frac {1}{2}}g(\\psi \\gamma ^{\\mu }\\psi )^{2}', 'id': '77e840f5bef1b29', 'link': 'https://ko.wikipedia.org/wiki/%ED%8B%B0%EB%A7%81_%EB%AA%A8%ED%98%95', 'title': '티링 모형'}}, {'_index': 'search_index', '_type': '_doc', '_id': 'fa9b06265c0f6c9', '_score': 3.2517695, '_source': {'latex': '\\mathbf {G} ={

/var/folders/3c/k281jc6s6yq4s3qndkn6tw880000gn/T/ipykernel_17248/639292111.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index=INDEX, body=body)
